# tRNA-seq with E.coli total RNA spike-in

*Isodecoders aggregated to isoacceptors*

All figures are in the "figs" folder.

DESeq result tables are in the "deseq_results" folder.

Remember to change "human", "Homo sapiens", paths to files etc. to match your data!

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns
from adjustText import adjust_text

In [ ]:
def rev_comp(anticodon):
    mapping = {'A':'T', 'T':'A', 'C':'G', 'G':'C'}
    res = ''.join([mapping.get(i, i) for i in anticodon[::-1]])
    return res

In [ ]:
def label_point(x, y, val, ax):
    a = pd.concat({'x': x, 'y': y, 'val': val}, axis=1)
    for i, point in a.iterrows():
        ax.text(point['x']+.02, point['y'], str(point['val']))

def plot_volcano(df, save=None, labels=True,xlim=None, ylim=None, figsize=(12,12),title=None):
    df_ns = df[df['padj'] >= 0.01]
    df_significant = df[df['padj'] < 0.01]
    plt.figure(figsize=figsize)
    g = sns.scatterplot(data=df_ns, x="log2FoldChange", y=-np.log10(df_ns['padj']), color='grey')
    h = sns.scatterplot(data=df_significant, x="log2FoldChange", y=-np.log10(df_significant['padj']), color='blue', s=80)
    # g.set(ylim=(-1,60))
    if xlim is not None:
        g.set(xlim=xlim)
    g.set(ylabel='-log10(Padj)')
    g.set(xlabel='log2FC')
    g.axvline(0, alpha=0.5)
    g.axhline(2, alpha=0.5, c='red')
    if labels is True:
        texts = []
        for x, y, s in zip(df_significant['log2FoldChange'].tolist(), (-np.log10(df_significant['padj'])).tolist(), df_significant['gene'].tolist()):
            texts.append(plt.text(x, y, s, fontsize=12))
        # label_point(df03b_significant['log2FoldChange'], -np.log10(df03b_significant['padj']), df03b_significant['gene'], h)
        adjust_text(texts, only_move={'points':'y', 'texts':'y'}, arrowprops=dict(arrowstyle="->", color='r', lw=0.5), expand_points=(2,2))
        for item in ([g.title, g.xaxis.label, g.yaxis.label] +
                     g.get_xticklabels() + g.get_yticklabels()):
            item.set_fontsize(15)
    sign = mpatches.Patch(color='blue', label="adjusted p < 0.01")
    notsign = mpatches.Patch(color='grey', label="ns")
    plt.legend(handles=[sign, notsign], fontsize=12)
    plt.suptitle(title)
    if save is not None:
        plt.savefig(save, dpi=300, bbox_inches='tight')
    plt.show()

Aggregate tRNAs by anticodons

In [ ]:
path = "03_featureCounts/"
out_list = []
for file in os.listdir(path):
    if "summary" not in file:
        df = pd.read_csv(path+file, sep='\t', skiprows=1).drop(['Chr', 'Start', 'End', 'Strand', 'Length'], axis=1)
        df = df.set_index('Geneid')
        out_list.append(df)
df00 = pd.concat(out_list, axis=1).reset_index()
df00['Geneid'] = df00['Geneid'].str.replace("-", "_")

In [ ]:
ss = df00['Geneid'].str.split("_tRNA_", expand=True)
df00['anticodon'] = ss[0] + "_tRNA_" + ss[1].str.split("_", expand=True)[0] + "_" + ss[1].str.split("_", expand=True)[1]

In [ ]:
df01 = df00.groupby('anticodon').sum().drop('Geneid',axis=1)
df01.to_csv("featureCounts_aggregated.csv")

### DESeq

In [ ]:
%load_ext rpy2.ipython

In [ ]:
%%R

library("DESeq2", quietly = T)
# library("tximport", quietly = T)
library("dplyr", quietly = T)
library("ggplot2", quietly = T)
library("purrr", quietly = T)
library("pheatmap", quietly = T)
library("RColorBrewer", quietly = T)

Load featureCounts results. The fastest way is to put them in a separate folder and then read all files in it:

In [ ]:
%%R

# files <- list.files("03_featureCounts", full.names = T)
# # Ignore the summary files
# files <- files[which(!grepl(".summary", files))]

# cts <- lapply(files, function(x) read.csv(x, sep='\t', skip=1, stringsAsFactors=F))
# Join all files into a single dataframe

cts = read.csv("featureCounts_aggregated.csv")

# by_cols <- colnames(cts[[1]])[1:6]
# cts <- cts %>% reduce(full_join, by = by_cols)
rownames(cts) <- cts$anticodon
# Drop unnecessary columns
drops <- c("anticodon")
cts <- cts[ , !(names(cts) %in% drops)]
# Clean up column names
colnames(cts) <- gsub("X02_alignment_bam.", "", colnames(cts))
colnames(cts)

Add condition (or cell line, treatment, etc.) information.

**IMPORTANT:** Make sure that samples and conditions are properly matched in the output below!

In [ ]:
%%R

# Add condition information
coldata <- data.frame(line = rep(c("WT","POLR2B_HF","POLR2B_HF_POLR3A_mut"),3), row.names = colnames(cts))
#                     ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# here ----------------------------^
coldata

Use E. coli genes to estimate size factors (this is the recommended method to use spike-ins in DESeq, as mentioned [here](https://support.bioconductor.org/p/9149000/#9149006)

In [ ]:
%%R

# Separate target and spike-in counts
cts_coli <- cts[which(grepl("Escherichia", rownames(cts))),]
cts_human <- cts[which(grepl("Homo", rownames(cts))),]
dds <- DESeqDataSetFromMatrix(countData = cts,
                              colData = coldata,
                              design = ~ line)

dds <- estimateSizeFactors(dds, controlGenes=which(grepl("Escherichia", rownames(cts))))
# keep <- rowSums(counts(dds)) > 1
# dds <- dds[keep,]
# dds_coli <-DESeqDataSetFromMatrix(countData = cts_coli,
#                                   colData = coldata,
#                                   design = ~ line)
# dds_coli <- estimateSizeFactors(dds_coli)
# dds_human <-DESeqDataSetFromMatrix(countData = cts_human,
#                                   colData = coldata,
#                                   design = ~ line)
# sizeFactors(dds_human) <- sizeFactors(dds_coli)
dds <- dds[which(grepl("Homo", rownames(cts))),]

Plot PCA and distance heatmap for all samples

In [ ]:
%%R

# PCA
rld <- rlog(dds, blind = FALSE)
pcaData <- plotPCA(rld, intgroup = c("line"), returnData = T)
percentVar <- round(100 * attr(pcaData, "percentVar"))
g = ggplot(pcaData, aes(x = PC1, y = PC2, color = line)) +
  geom_point(size =3) +
  xlab(paste0("PC1: ", percentVar[1], "% variance")) +
  ylab(paste0("PC2: ", percentVar[2], "% variance")) +
  coord_fixed() +
  ggtitle("PCA with rlog data, aggregated before DESeq")
ggsave("figs/aggregated_PCA.png")
print(g)

In [ ]:
%%R

sampleDists <- dist(t(assay(rld)))
sampleDistMatrix <- as.matrix(sampleDists)
rownames(sampleDistMatrix) <- rld$line
colnames(sampleDistMatrix) <- NULL
colors <- colorRampPalette( rev(brewer.pal(9, "Blues")) )(255)
g = pheatmap(sampleDistMatrix,
         clustering_distance_rows=sampleDists,
         clustering_distance_cols=sampleDists,
         col=colors)
ggsave("figs/aggregated_clustermap.png", plot=g)
print(g)

Run DESeq analysis

In [ ]:
%%R

deseq <- DESeq(dds)
# deseq_lrt <- DESeq(dds, test = "LRT", reduced= ~1)

Write result tables (make sure the destination directory exists)

In [ ]:
%%R

res <- results(deseq, contrast = c("line", "POLR2B_HF", "WT"))
write.csv(res, "deseq_results/aggregated_deseq_FC_q10_K562_tRNAseq_POLR2B_HF_vs_WT.csv")
plotMA(res)

In [ ]:
%%R

res <- results(deseq, contrast = c("line", "POLR2B_HF_POLR3A_mut", "POLR2B_HF"))
write.csv(res, "deseq_results/aggregated_deseq_FC_q10_K562_tRNAseq_mutant_vs_POLR2B_HF.csv")
plotMA(res)

### Plotting results

In [ ]:
df00

In [ ]:
df00 = pd.read_csv("deseq_results/aggregated_deseq_FC_q10_K562_tRNAseq_POLR2B_HF_vs_WT.csv")
df00.rename({'Unnamed: 0': 'gene'}, axis=1, inplace=True)
df00['gene'] = df00['gene'].str.replace("Homo_sapiens_tRNA_", "")
df00['gene'] = df00['gene'].str.replace(" ", "")
df01_sig = df00[df00['padj'] < 0.1]
df01_sig.sort_values('log2FoldChange', ascending=False)

In [ ]:
plot_volcano(df00, labels=False, save="figs/aggregated_volcano_POLR2b_HF_vs_WT", title="tRNAseq aggregated_DE: POLR2B-HF vs WT")

In [ ]:
plt.figure(figsize=(12,5))
plt.xticks(rotation=90, ha='center')
sns.barplot(data=df01_sig.sort_values('gene', ascending=False), x='gene', y='log2FoldChange', palette='summer')
plt.suptitle("POLR2B-HF vs WT: Significantly aggregated_DE isodecoders")
plt.savefig("figs/aggregated_K562_tRNAseq_POLR2B_HF_vs_WT_isodecoders.png", dpi=300, bbox_inches='tight')

In [ ]:
df02 = pd.read_csv("deseq_results/aggregated_deseq_FC_q10_K562_tRNAseq_mutant_vs_POLR2B_HF.csv")
df02.rename({'Unnamed: 0': 'gene'}, axis=1, inplace=True)
df02['gene'] = df02['gene'].str.replace("Homo_sapiens_tRNA_", "")
df02['gene'] = df02['gene'].str.replace(" ", "")
df03_sig = df02[df02['padj'] < 0.01]
df03_sig.sort_values('log2FoldChange', ascending=False)

In [ ]:
plot_volcano(df02, labels=True,save="figs/aggregated_volcano_mutant_vs_POLR2b_HF", title="tRNAseq aggregated_DE: Mutant vs POLR2B-HF")

In [ ]:
plt.figure(figsize=(12,5))
plt.xticks(rotation=90, ha='center')
sns.barplot(data=df03_sig.sort_values('gene', ascending=False), x='gene', y='log2FoldChange', palette='summer')
plt.suptitle("mutant vs POLR2B-HF: Significantly aggregated_DE isodecoders")
plt.savefig("figs/aggregated_mutant_vs_POLR2b_HF_isodecoders_bar.png", dpi=300, bbox_inches='tight')